[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ghodsizadeh/jalali-pandas/blob/main/examples/00_zero_to_hero.ipynb)
[![PyPI version](https://img.shields.io/pypi/v/jalali-pandas.svg)](https://pypi.org/project/jalali-pandas/)
[![PyPI downloads](https://img.shields.io/pypi/dm/jalali-pandas.svg)](https://pypi.org/project/jalali-pandas/)
[![Python versions](https://img.shields.io/pypi/pyversions/jalali-pandas.svg)](https://pypi.org/project/jalali-pandas/)
[![License](https://img.shields.io/badge/License-GPLv3-blue.svg)](https://www.gnu.org/licenses/gpl-3.0)
[![Coverage](https://codecov.io/gh/ghodsizadeh/jalali-pandas/branch/main/graph/badge.svg?token=LWQ85TN0NU)](https://codecov.io/gh/ghodsizadeh/jalali-pandas)
![GitHub Repo stars](https://img.shields.io/github/stars/ghodsizadeh/jalali-pandas?logoColor=blue&style=social)

# Jalali Pandas: Zero to Hero
Full Jalali (Persian/Shamsi) calendar support for pandas, with native dtypes, indexes, offsets, and time-series operations.

## Notebook README

This notebook is a hands-on guide to the full Jalali Pandas API, from basic
conversions to Jalali-aware resampling and offsets. It is designed to be read
top-to-bottom, but each section works as a standalone reference.

**You will learn**
- Convert between Gregorian and Jalali dates (scalar + vectorized).
- Use `JalaliTimestamp`, the Jalali dtype, and `JalaliDatetimeIndex`.
- Generate Jalali date ranges and apply Jalali offsets.
- Use the Series/DataFrame `.jalali` accessors.
- Group and resample by Jalali calendar boundaries.

**Requirements**
- Python 3.9+
- pandas
- jdatetime (installed automatically with `jalali-pandas`)

**Project links**
- GitHub: https://github.com/ghodsizadeh/jalali-pandas
- PyPI: https://pypi.org/project/jalali-pandas/

**Contents**
1. Install and import
2. Quickstart with a demo DataFrame
3. JalaliTimestamp (core type)
4. Vectorized conversions and dtype
5. JalaliDatetimeIndex and date ranges
6. Offsets and frequency aliases
7. Series accessor (jalali)
8. DataFrame accessor (jalali)
9. Jalali groupby and resample on a Gregorian index
10. End-to-end pipeline and tips

## 1. Install
Run this once in a fresh environment (Colab, new venv). Skip if already installed.

In [1]:
# If you're running this in Colab or a fresh environment, install first:
# !pip -q install -U jalali-pandas

# For the latest main branch (optional):
# !pip -q install -U git+https://github.com/ghodsizadeh/jalali-pandas.git

## 2. Imports and setup

In [2]:
import numpy as np
import pandas as pd

import jalali_pandas  # registers accessors: Series.jalali and DataFrame.jalali
from jalali_pandas import (
    JalaliDatetimeDtype,
    JalaliDatetimeIndex,
    JalaliTimestamp,
    days_in_month,
    days_in_year,
    is_leap_year,
    jalali_date_range,
    to_gregorian_datetime,
    to_jalali_datetime,
)
from jalali_pandas.api import JalaliGrouper, resample_jalali
from jalali_pandas.offsets import (
    FRIDAY,
    JalaliMonthBegin,
    JalaliMonthEnd,
    JalaliQuarterBegin,
    JalaliQuarterEnd,
    JalaliWeek,
    JalaliYearBegin,
    JalaliYearEnd,
    list_jalali_aliases,
    parse_jalali_frequency,
)

pd.set_option("display.width", 120)
pd.set_option("display.max_columns", 20)

### Quick sanity check

In [3]:
jalali_pandas.__version__

'1.0.0a1'

## 3. Quickstart: build a demo DataFrame (Gregorian)
We start with a normal pandas DataFrame in Gregorian dates.

In [4]:
rng = pd.date_range("2023-03-01", periods=120, freq="D")
rng.name = "date"

gen = np.random.default_rng(42)
df = pd.DataFrame(
    {
        "date": rng,
        "sales": gen.integers(10, 120, size=len(rng)),
        "orders": gen.integers(1, 25, size=len(rng)),
        "channel": np.where(np.arange(len(rng)) % 2 == 0, "online", "store"),
    }
)

df.head()

,date,sales,orders,channel
0,2023-03-01,19,11,online
1,2023-03-02,95,17,store
2,2023-03-03,82,16,online
3,2023-03-04,58,12,store
4,2023-03-05,57,21,online


### Convert to Jalali (Series accessor)
`Series.jalali.to_jalali()` converts Gregorian timestamps to `jdatetime` objects.

In [5]:
df["jdate"] = df["date"].jalali.to_jalali()
df[["date", "jdate"]].head()

,date,jdate
0,2023-03-01,1401-12-10 00:00:00
1,2023-03-02,1401-12-11 00:00:00
2,2023-03-03,1401-12-12 00:00:00
3,2023-03-04,1401-12-13 00:00:00
4,2023-03-05,1401-12-14 00:00:00


## 4. Series accessor: parsing, conversion, and properties
The `.jalali` accessor works on Series of jdatetime objects or Jalali strings.

In [6]:
jstrings = pd.Series(["1402-01-01", "1402-01-02", None, "1402-01-04"])
jparsed = jstrings.jalali.parse_jalali("%Y-%m-%d")

pd.DataFrame(
    {
        "raw": jstrings,
        "parsed": jparsed,
        "to_gregorian": jparsed.jalali.to_gregorian(),
    }
)

,raw,parsed,to_gregorian
0,1402-01-01,1402-01-01 00:00:00,2023-03-21
1,1402-01-02,1402-01-02 00:00:00,2023-03-22
2,None,NaT,NaT
3,1402-01-04,1402-01-04 00:00:00,2023-03-24


### Vectorized Jalali properties
Get year/month/day, week, day-of-year, and boolean flags.

In [7]:
df["jyear"] = df["jdate"].jalali.year
df["jmonth"] = df["jdate"].jalali.month
df["jday"] = df["jdate"].jalali.day
df["jweek"] = df["jdate"].jalali.week
df["jweekday"] = df["jdate"].jalali.weekday
df["jdayofyear"] = df["jdate"].jalali.dayofyear
df["is_month_end"] = df["jdate"].jalali.is_month_end

df[
    [
        "jdate",
        "jyear",
        "jmonth",
        "jday",
        "jweek",
        "jweekday",
        "jdayofyear",
        "is_month_end",
    ]
].head()

,jdate,jyear,jmonth,jday,jweek,jweekday,jdayofyear,is_month_end
0,1401-12-10 00:00:00,1401,12,10,50,4,346,False
1,1401-12-11 00:00:00,1401,12,11,50,5,347,False
2,1401-12-12 00:00:00,1401,12,12,50,6,348,False
3,1401-12-13 00:00:00,1401,12,13,50,0,349,False
4,1401-12-14 00:00:00,1401,12,14,50,1,350,False


### Formatting and names
Use `strftime`, `month_name`, and `day_name` (supports `locale="fa"` or `"en"`).

In [8]:
pd.DataFrame(
    {
        "jdate": df["jdate"].head(),
        "formatted": df["jdate"].jalali.strftime("%Y/%m/%d").head(),
        "month_name": df["jdate"].jalali.month_name().head(),
        "day_name": df["jdate"].jalali.day_name().head(),
    }
)

,jdate,formatted,month_name,day_name
0,1401-12-10 00:00:00,1401/12/10,Esfand,Doshanbeh
1,1401-12-11 00:00:00,1401/12/11,Esfand,Seshanbeh
2,1401-12-12 00:00:00,1401/12/12,Esfand,Chaharshanbeh
3,1401-12-13 00:00:00,1401/12/13,Esfand,Panjshanbeh
4,1401-12-14 00:00:00,1401/12/14,Esfand,Jomeh


### Normalize, floor, ceil, round (Series)
These operate on `jdatetime.datetime` values and return jdatetime objects.

In [9]:
time_rng = pd.date_range("2023-03-01 08:15", periods=5, freq="7H")
time_series = pd.Series(time_rng, name="ts")
jtime = time_series.jalali.to_jalali()

pd.DataFrame(
    {
        "orig": jtime,
        "normalize": jtime.jalali.normalize(),
        "floor_hour": jtime.jalali.floor("h"),
        "ceil_hour": jtime.jalali.ceil("h"),
        "round_hour": jtime.jalali.round("h"),
    }
)

/var/folders/0s/wxqxsdh57jq6sxnr8fj3j5080000gn/T/ipykernel_42563/1793260748.py:1: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  time_rng = pd.date_range("2023-03-01 08:15", periods=5, freq="7H")


,orig,normalize,floor_hour,ceil_hour,round_hour
0,1401-12-10 08:15:00,1401-12-10 00:00:00,1401-12-10 08:00:00,1401-12-10 09:00:00,1401-12-10 08:00:00
1,1401-12-10 15:15:00,1401-12-10 00:00:00,1401-12-10 15:00:00,1401-12-10 16:00:00,1401-12-10 15:00:00
2,1401-12-10 22:15:00,1401-12-10 00:00:00,1401-12-10 22:00:00,1401-12-10 23:00:00,1401-12-10 22:00:00
3,1401-12-11 05:15:00,1401-12-11 00:00:00,1401-12-11 05:00:00,1401-12-11 06:00:00,1401-12-11 05:00:00
4,1401-12-11 12:15:00,1401-12-11 00:00:00,1401-12-11 12:00:00,1401-12-11 13:00:00,1401-12-11 12:00:00


### Timezone helpers (Series)
`tz_localize` and `tz_convert` return **Gregorian** timestamps with timezone info.

In [10]:
jtime_utc = jtime.jalali.tz_localize("UTC")
jtime_utc.head()

0   2023-03-01 08:15:00+00:00
1   2023-03-01 15:15:00+00:00
2   2023-03-01 22:15:00+00:00
3   2023-03-02 05:15:00+00:00
4   2023-03-02 12:15:00+00:00
Name: ts, dtype: datetime64[ns, UTC]

## 5. Core type: JalaliTimestamp
`JalaliTimestamp` is the scalar Jalali datetime, similar to `pd.Timestamp`.

In [11]:
ts = JalaliTimestamp(1402, 6, 15, 14, 30, 45)
ts

JalaliTimestamp('1402-06-15T14:30:45')

In [12]:
pd.Series(
    {
        "year": ts.year,
        "month": ts.month,
        "day": ts.day,
        "quarter": ts.quarter,
        "weekday": ts.weekday,
        "dayofyear": ts.dayofyear,
        "is_leap_year": ts.is_leap_year,
        "gregorian": ts.to_gregorian(),
        "normalize": ts.normalize(),
        "replace_month": ts.replace(month=1, day=1),
    }
)

year                            1402
month                              6
day                               15
quarter                            2
weekday                            2
dayofyear                        170
is_leap_year                   False
gregorian        2023-09-06 14:30:45
normalize        1402-06-15 00:00:00
replace_month    1402-01-01 14:30:45
dtype: object

In [13]:
ts_plus = ts + pd.Timedelta(days=10)
ts_minus = ts - pd.Timedelta(days=7)
diff = ts_plus - ts

pd.Series(
    {
        "ts": ts,
        "ts_plus_10d": ts_plus,
        "ts_minus_7d": ts_minus,
        "diff": diff,
    }
)

ts             1402-06-15 14:30:45
ts_plus_10d    1402-06-25 14:30:45
ts_minus_7d    1402-06-08 14:30:45
diff              10 days 00:00:00
dtype: object

### Calendar utilities
Quick helpers for leap-year and month/year length checks.

In [14]:
pd.Series(
    {
        "is_leap_1403": is_leap_year(1403),
        "days_in_1402_12": days_in_month(1402, 12),
        "days_in_1403": days_in_year(1403),
    }
)

is_leap_1403       True
days_in_1402_12      29
days_in_1403        366
dtype: object

## 6. Vectorized conversion API and Jalali dtype
Use `to_jalali_datetime` and `to_gregorian_datetime` for arrays, indexes, or Series.

In [15]:
j_from_strings = to_jalali_datetime(["1402-01-01", "1402-01-02", "1402-01-03"])
j_from_strings

JalaliDatetimeIndex(['1402-01-01', '1402-01-02', '1402-01-03'], dtype='jalali_datetime', freq=None)

In [16]:
jalali_typed = pd.Series(
    [JalaliTimestamp(1402, 1, 1), JalaliTimestamp(1402, 1, 2), pd.NaT],
    dtype=JalaliDatetimeDtype(),
)

pd.Series({"dtype": str(jalali_typed.dtype), "values": str(jalali_typed.tolist())})

dtype                                       jalali_datetime
values    [JalaliTimestamp('1402-01-01T00:00:00'), Jalal...
dtype: object

In [17]:
greg_index = pd.date_range("2023-03-21", periods=4, freq="D")
j_index = to_jalali_datetime(greg_index)

pd.Series(
    {
        "jalali_index": str(j_index),
        "dtype": str(j_index.dtype),
        "back_to_gregorian": str(to_gregorian_datetime(j_index)),
    }
)

jalali_index         JalaliDatetimeIndex(['1402-01-01', '1402-01-02...
dtype                                                  jalali_datetime
back_to_gregorian    DatetimeIndex(['2023-03-21', '2023-03-22', '20...
dtype: object

In [18]:
greg_series = pd.Series(pd.date_range("2023-03-21", periods=4, freq="D"))
jalali_series = to_jalali_datetime(greg_series)

pd.Series(
    {
        "series_dtype": str(jalali_series.dtype),
        "head": str(jalali_series.head(2).tolist()),
    }
)

series_dtype                                      jalali_datetime
head            [JalaliTimestamp('1402-01-01T00:00:00'), Jalal...
dtype: object

## 7. JalaliDatetimeIndex and date ranges
Create Jalali-native ranges and shift/snap them with Jalali offsets.

In [19]:
JalaliDatetimeIndex(["1402-01-01", "1402-01-02", "1402-01-03"])

JalaliDatetimeIndex(['1402-01-01', '1402-01-02', '1402-01-03'], dtype='jalali_datetime', freq=None)

In [20]:
jrange = jalali_date_range("1402-01-01", periods=6, freq="D")
jrange

JalaliDatetimeIndex(['1402-01-01', '1402-01-02', '1402-01-03', '1402-01-04', '1402-01-05', ...], dtype='jalali_datetime', freq='D')

In [21]:
pd.Series(
    {
        "freq": jrange.freqstr,
        "inferred_freq": jrange.inferred_freq,
        "to_gregorian": str(jrange.to_gregorian()),
    }
)

freq                                                             D
inferred_freq                                                    D
to_gregorian     DatetimeIndex(['2023-03-21', '2023-03-22', '20...
dtype: object

In [22]:
jrange.shift(1, freq="JME")

JalaliDatetimeIndex(['1402-02-31', '1402-02-31', '1402-02-31', '1402-02-31', '1402-02-31', ...], dtype='jalali_datetime', freq='D')

## 8. Jalali offsets and frequency aliases
Offsets respect Jalali calendar boundaries (month, quarter, year, week).

In [23]:
list_jalali_aliases()

{'JME': 'JalaliMonthEnd',
 'JMS': 'JalaliMonthBegin',
 'JQE': 'JalaliQuarterEnd',
 'JQS': 'JalaliQuarterBegin',
 'JYE': 'JalaliYearEnd',
 'JYS': 'JalaliYearBegin',
 'JW': 'JalaliWeek'}

In [24]:
ts = JalaliTimestamp(1402, 6, 15)

pd.Series(
    {
        "JME": (JalaliMonthEnd() + ts),
        "JMS": (JalaliMonthBegin() + ts),
        "JQE": (JalaliQuarterEnd() + ts),
        "JQS": (JalaliQuarterBegin() + ts),
        "JYE": (JalaliYearEnd() + ts),
        "JYS": (JalaliYearBegin() + ts),
        "JW (Friday)": (JalaliWeek(weekday=FRIDAY) + ts),
    }
)

JME            1402-07-30 00:00:00
JMS            1402-07-01 00:00:00
JQE            1402-09-30 00:00:00
JQS            1402-07-01 00:00:00
JYE            1403-12-30 00:00:00
JYS            1403-01-01 00:00:00
JW (Friday)    1402-06-19 00:00:00
dtype: object

In [25]:
parse_jalali_frequency("2JME")

<JalaliMonthEnd: n=2>

## 9. DataFrame accessor: groupby, resample, convert, filter
`df.jalali` expects a column with jdatetime values (we use `jdate`).

In [26]:
# Group by Jalali year + month
df.jalali.groupby("ym").sum(numeric_only=True).head()

sales  orders  jyear  jmonth  jday  jweek  jweekday  jdayofyear
__year __month                                                                 
1401   12        1374     251  28020     240   390   1024        60        7110
1402   1         2172     374  43462      31   496     89        96         496
       2         1962     393  43462      62   496    225        91        1457
       3         2202     384  43462      93   496    364        93        2418
       4          295     100   9814      28    28    101        21         679

In [27]:
# Resample by Jalali month
df.jalali.resample("month").head()

,sales,orders,jyear,jmonth,jday,jweek,jweekday,jdayofyear
0,1374,251,28020,240,390,1024,60,7110
1,2172,374,43462,31,496,89,96,496
2,1962,393,43462,62,496,225,91,1457
3,2202,384,43462,93,496,364,93,2418
4,295,100,9814,28,28,101,21,679


### Convert columns and create Jalali period labels

In [28]:
df_converted = df.jalali.convert_columns("date", to_jalali=True)
df_converted[["date", "jdate"]].head()

,date,jdate
0,1401-12-10 00:00:00,1401-12-10 00:00:00
1,1401-12-11 00:00:00,1401-12-11 00:00:00
2,1401-12-12 00:00:00,1401-12-12 00:00:00
3,1401-12-13 00:00:00,1401-12-13 00:00:00
4,1401-12-14 00:00:00,1401-12-14 00:00:00


In [29]:
df.jalali.to_period("M").head()

,date,sales,orders,channel,jdate,jyear,jmonth,jday,jweek,jweekday,jdayofyear,is_month_end,jdate_period
0,2023-03-01,19,11,online,1401-12-10 00:00:00,1401,12,10,50,4,346,False,1401-12
1,2023-03-02,95,17,store,1401-12-11 00:00:00,1401,12,11,50,5,347,False,1401-12
2,2023-03-03,82,16,online,1401-12-12 00:00:00,1401,12,12,50,6,348,False,1401-12
3,2023-03-04,58,12,store,1401-12-13 00:00:00,1401,12,13,50,0,349,False,1401-12
4,2023-03-05,57,21,online,1401-12-14 00:00:00,1401,12,14,50,1,350,False,1401-12


### Filter by Jalali date components

In [30]:
df_1402 = df.jalali.filter_by_year(1402)
df_1402.head()

,date,sales,orders,channel,jdate,jyear,jmonth,jday,jweek,jweekday,jdayofyear,is_month_end
20,2023-03-21,65,24,online,1402-01-01 00:00:00,1402,1,1,1,3,1,False
21,2023-03-22,50,6,store,1402-01-02 00:00:00,1402,1,2,1,4,2,False
22,2023-03-23,30,7,online,1402-01-03 00:00:00,1402,1,3,1,5,3,False
23,2023-03-24,111,10,store,1402-01-04 00:00:00,1402,1,4,1,6,4,False
24,2023-03-25,95,24,online,1402-01-05 00:00:00,1402,1,5,1,0,5,False


In [31]:
df.jalali.filter_by_date_range(start="1402-01-15", end="1402-02-15").head()

,date,sales,orders,channel,jdate,jyear,jmonth,jday,jweek,jweekday,jdayofyear,is_month_end
34,2023-04-04,107,11,online,1402-01-15 00:00:00,1402,1,15,3,3,15,False
35,2023-04-05,17,16,store,1402-01-16 00:00:00,1402,1,16,3,4,16,False
36,2023-04-06,104,4,online,1402-01-17 00:00:00,1402,1,17,3,5,17,False
37,2023-04-07,101,14,store,1402-01-18 00:00:00,1402,1,18,3,6,18,False
38,2023-04-08,40,13,online,1402-01-19 00:00:00,1402,1,19,3,0,19,False


## 10. Jalali groupby/resample on a Gregorian index
You can group or resample by Jalali boundaries without a `jdate` column.

Note: `jalali_groupby` exists as a convenience wrapper around `JalaliGrouper`.
If you hit compatibility issues, use `JalaliGrouper.get_grouper()` directly.

In [32]:
# JalaliGrouper: group by Jalali month end based on a Gregorian column
month_grouper = JalaliGrouper(key="date", freq="JME")
df.groupby(month_grouper.get_grouper(df)).sum(numeric_only=True).head()

,sales,orders,jyear,jmonth,jday,jweek,jweekday,jdayofyear,is_month_end
2023-03-20,1374,251,28020,240,390,1024,60,7110,1
2023-04-20,2172,374,43462,31,496,89,96,496,1
2023-05-21,1962,393,43462,62,496,225,91,1457,1
2023-06-21,2202,384,43462,93,496,364,93,2418,1
2023-07-22,295,100,9814,28,28,101,21,679,0


In [33]:
# Jalali resampling works on DatetimeIndex
df_indexed = df.set_index("date")
resample_jalali(df_indexed["sales"], "JME").sum().head()

2023-03-20    1374
2023-04-20    2172
2023-05-21    1962
2023-06-21    2202
2023-07-22     295
Name: sales, dtype: int64

## 11. End-to-end pipeline: a Jalali calendar report
Build a monthly Jalali report from raw Gregorian data in a few steps.

In [43]:
raw = pd.DataFrame(
    {
        "date": pd.date_range("2023-01-01", periods=180, freq="D"),
        "revenue": gen.integers(50, 400, size=180),
        "returns": gen.integers(0, 30, size=180),
    }
)

raw["jdate"] = raw["date"].jalali.to_jalali()
raw["jmonth"] = raw["jdate"].jalali.month
raw["jyear"] = raw["jdate"].jalali.year

report = (raw.jalali.groupby("ym").sum().rename_axis(index=["jyear", "jmonth"]))[
    ["revenue", "returns"]
]

report.head()

revenue  returns
jyear jmonth                  
1401  10         4289      280
      11         6070      419
      12         5970      451
1402  1          7532      507
      2          6648      375

## 12. Tips and gotchas
- The `.jalali` accessor is registered when you `import jalali_pandas`.
- `Series.jalali.tz_localize`/`tz_convert` return **Gregorian** datetimes.
- Use `to_jalali_datetime` if you need the Jalali extension dtype.
- For Jalali calendar grouping on Gregorian indexes, prefer `JalaliGrouper` or `resample_jalali`.